<a href="https://colab.research.google.com/github/rabinkahoonay/test/blob/main/project_capital_markets_24_3_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#!pip install yfinance --upgrade --no-cache-dir
#!pip install yahoo_fin
#!mkdir -p '/content/drive/My Drive/Colab_Data'
import pandas as pd
import yfinance as yf
import calendar
from datetime import date
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from google.colab import drive
#drive.mount('/content/drive')
df_h = pd.read_csv('/content/drive/My Drive/Colab_Data/data.csv')
MAX_DATE=df_h.index.max()
print(MAX_DATE)
df_h = df_h.drop(df_h.index[df_h.index == df_h.index.max()])
type (df_h)

11669


pandas.core.frame.DataFrame

In [14]:

index_list = ['^GSPC', '^TA125.TA', '^DJI', '^IXIC',
              '^VIX','^VVIX','^VIX9D','^VIX3M','^VIX6M','^VIX1Y',
              '^IRX','^FVX','^TNX','^TYX']
#data = yf.download(index_list,'1980-1-1')['Adj Close']
data = yf.download(index_list,MAX_DATE)['Adj Close']
#pd=pd.set_option('display.max_columns', None)
df= pd.DataFrame([[data]])
df=data.fillna(method='ffill')
df = df.reset_index().round(2)
df = df.rename(columns={  '^GSPC' :'S&P500'    ,'^TA125.TA':'TA_125',
                          '^DJI'  :'Dow_Jones' ,'^IXIC'    :'NASDAQ',
                          '^VIX'  :'VIX'       ,'^VVIX'    :'VIX_Index',
                          '^VIX9D':'VIX9D'     ,'^VIX3M'   :'VIX3M',
                          '^VIX6M':'VIX6M'     ,'^VIX1Y'   :'VIX1Y',
                          '^IRX'  :'TB_13W'    ,'^FVX'     :'T5Y',
                          '^TNX'  :'T10Y'      ,'^TYX'     :'T30Y'
                          })
# limit to 2 digits after the decimal point
#daily_data=df.round(2)
# print ("Daily Data of the indices \n   " , daily_data)

df['Day'] = pd.to_datetime(df['Date']).dt.day
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Year'] = pd.to_datetime(df['Date']).dt.year
df = df.set_index('Date')
df = pd.concat([df_h, df], axis=1)
df.to_csv('/content/drive/My Drive/Colab_Data/data.csv')

[*********************100%%**********************]  14 of 14 completed


In [15]:
def add_daily_change(df):
  # Identify time series columns (assuming numeric datatypes for simplicity)
  numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
  timeseries_cols = [col for col in numeric_cols if len(df) > 1]  # Ensure there are at least 2 rows

  # Create new columns with daily changes (shift by -1 for difference)
  for col in timeseries_cols:
    df[col + "_p"] = df[col].diff(periods=1).round(2)
    df[col + "_c1"]= (df[col].pct_change(1)*100).round(2)
    df[col + "_c2"]= (df[col].pct_change(2)*100).round(2)
    df[col + "_c3"]= (df[col].pct_change(3)*100).round(2)
  return df



df = add_daily_change(df.copy())  # Operate on a copy to avoid modifying the original
df=df.round(2)
#print("Daily Data of the indices \n   " , df.tail(5))
df.tail(5).T

,2024-03-18 00:00:00,2024-03-19 00:00:00,2024-03-20 00:00:00,2024-03-21 00:00:00,2024-03-22 00:00:00
Date,NaN,NaN,NaN,NaN,NaN
Dow_Jones,NaN,NaN,NaN,NaN,NaN
T5Y,NaN,NaN,NaN,NaN,NaN
S&P500,NaN,NaN,NaN,NaN,NaN
TB_13W,NaN,NaN,NaN,NaN,NaN
NASDAQ,NaN,NaN,NaN,NaN,NaN
TA_125,NaN,NaN,NaN,NaN,NaN
T10Y,NaN,NaN,NaN,NaN,NaN
T30Y,NaN,NaN,NaN,NaN,NaN
VIX,NaN,NaN,NaN,NaN,NaN


In [16]:
# חישוב ממוצע נע עבור עמדות ספציפיות
New_Names=['S&P500','TA_125','Dow_Jones','NASDAQ','VIX','VIX_Index','VIX9D','VIX3M','VIX6M','VIX1Y','TB_13W','T5Y','T10Y','T30Y']
window_size = 30
for col in New_Names:
    df[col + "_MA30"] = df[col].rolling(window_size).mean().round(2)
    df[col + "/_MA30"] = (df[col]/df[col + "_MA30"]).round(4)
df.tail(5).T

ValueError: Cannot set a DataFrame with multiple columns to the single column S&P500_MA30

In [ ]:
import matplotlib.pyplot as plt
dates = pd.date_range("2022-01-01", pd.to_datetime("today"))
df1 = df.loc[df.index.isin(dates)]
plt.xlabel('Date')
plt.ylabel('T10Y')
plt.plot(df1.index, df1["T10Y"], linestyle="-", label="T10Y")
plt.plot(df1.index, df1["T10Y_MA30"], linestyle="--", label="T10Y (30-day MA)")
plt.legend()
plt.title('Time Series Plot of T10Y And T10Y_MA30')
plt.grid(True)

In [ ]:
! pip install TA-Lib
import talib as
rsi = talib.RSI(df[New_Names], timeperiod=14)

print(rsi)